# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
!nvidia-smi

Wed Jan  5 09:44:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [41]:
#!unzip /content/drive/MyDrive/Project_data.zip

In [4]:
import numpy as np
import os
#from scipy.misc import imread, imresize
from imageio import imread
from skimage.transform import resize as imresize
#import image as im
import datetime
import os
import PIL
import cv2 
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

We set the random seed so that the results don't vary drastically.

In [5]:
np.random.seed(30)
import random as rn
rn.seed(30)
import keras as Keras
from keras import backend as K
import tensorflow as tf
#tf.set_random_seed(30)
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [6]:
train_doc = np.random.permutation(open('/content/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/Project_data/val.csv').readlines())
batch_size = 10

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [7]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [8]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/Project_data/train'
val_path = '/content/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [22]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras

#write your model here
#model a
model_a = Sequential()

model_a.add(Conv3D(8, #number of filters 
                 kernel_size=(3,3,3), 
                 input_shape=(30, 120, 120, 3),
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(16, #Number of filters, 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(32, #Number of filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(64, #Number pf filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_a.add(Flatten())

model_a.add(Dense(1000, activation='relu'))
model_a.add(Dropout(0.5))

model_a.add(Dense(500, activation='relu'))
model_a.add(Dropout(0.5))

#softmax layer
model_a.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [10]:
optimiser = tf.keras.optimizers.Adam(lr=0.001) #write your optimizer
model_a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_a.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization (BatchN  (None, 30, 120, 120, 8)  32        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d (MaxPooling3D  (None, 15, 60, 60, 8)    0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_1 (Batc  (None, 15, 60, 60, 16)   6

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [11]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [12]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [13]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model_a.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 4.4519 - categorical_accuracy: 0.2881Source path =  /content/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-01-0414_50_06.200503/model-00001-4.45187-0.28808-1.21259-0.45000.h5
67/67 [==============================] - 279s 4s/step - loss: 4.4519 - categorical_accuracy: 0.2881 - val_loss: 1.2126 - val_categorical_accuracy: 0.4500 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 2.3758 - categorical_accuracy: 0.3383
Epoch 00002: saving model to model_init_2022-01-0414_50_06.200503/model-00002-2.37584-0.33831-1.54641-0.36000.h5
67/67 [==============================] - 107s 2s/step - loss: 2.3758 - categorical_accuracy: 0.3383 - val_loss: 1.5464 - val_categorical_accuracy: 0.3600 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.9715 - categorical_accuracy: 0.3035
Ep

In [14]:
classes = 5 #left swipe, right swipe, thumbs up, thumbs down, stop
channel = 3
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator_ex(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    batch_data[folder,idx] = temp #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx] = temp
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [ ]:
from keras.losses import categorical_crossentropy

# Define model b
model_b = Sequential()
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Flatten())
model_b.add(Dense(512, activation='relu'))
model_b.add(Dropout(0.5))
model_b.add(Dense(classes, activation='softmax'))

model_b.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_8 (Conv3D)           (None, 30, 120, 120, 32)  2624      
                                                                 
 activation_8 (Activation)   (None, 30, 120, 120, 32)  0         
                                                                 
 conv3d_9 (Conv3D)           (None, 30, 120, 120, 32)  27680     
                                                                 
 activation_9 (Activation)   (None, 30, 120, 120, 32)  0         
                                                                 
 max_pooling3d_8 (MaxPooling  (None, 10, 40, 40, 32)   0         
 3D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 10, 40, 40, 32)    0         
                                                      

In [ ]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.6622 - categorical_accuracy: 0.1825Source path =  /content/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2021-12-2910_13_59.732121/model-00001-1.66222-0.18250-1.60990-0.18000.h5
67/67 [==============================] - 331s 5s/step - loss: 1.6622 - categorical_accuracy: 0.1825 - val_loss: 1.6099 - val_categorical_accuracy: 0.1800 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.6105 - categorical_accuracy: 0.1741
Epoch 00002: saving model to model_init_2021-12-2910_13_59.732121/model-00002-1.61047-0.17413-1.60659-0.21000.h5
67/67 [==============================] - 125s 2s/step - loss: 1.6105 - categorical_accuracy: 0.1741 - val_loss: 1.6066 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.6083 - categorical_accuracy: 0.1542
Ep

### Changing x,y,z values Experiment [ 1]


In [ ]:
x = 30 # number of frames
y = 60 # image width
z = 60 # image height

In [ ]:
# Define model b
model_b = Sequential()
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))  
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Flatten())
model_b.add(Dense(512, activation='relu'))
model_b.add(Dropout(0.5))
model_b.add(Dense(classes, activation='softmax'))

model_b.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_12 (Conv3D)          (None, 30, 60, 60, 32)    2624      
                                                                 
 activation_12 (Activation)  (None, 30, 60, 60, 32)    0         
                                                                 
 conv3d_13 (Conv3D)          (None, 30, 60, 60, 32)    27680     
                                                                 
 activation_13 (Activation)  (None, 30, 60, 60, 32)    0         
                                                                 
 max_pooling3d_10 (MaxPoolin  (None, 10, 20, 20, 32)   0         
 g3D)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 10, 20, 20, 32)    0         
                                                      

In [ ]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.6365 - categorical_accuracy: 0.2368Source path =  /content/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2021-12-2910_13_59.732121/model-00001-1.63648-0.23680-1.53127-0.34000.h5
67/67 [==============================] - 292s 4s/step - loss: 1.6365 - categorical_accuracy: 0.2368 - val_loss: 1.5313 - val_categorical_accuracy: 0.3400 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.5672 - categorical_accuracy: 0.3035
Epoch 00002: saving model to model_init_2021-12-2910_13_59.732121/model-00002-1.56725-0.30348-1.36998-0.41000.h5
67/67 [==============================] - 120s 2s/step - loss: 1.5672 - categorical_accuracy: 0.3035 - val_loss: 1.3700 - val_categorical_accuracy: 0.4100 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.4206 - categorical_accuracy: 0.4030
Ep

### Changing Batch size  to 20 Experiment [ 2]


In [ ]:
train_generator = generator_ex(train_path, train_doc, 20)
val_generator = generator_ex(val_path, val_doc, 20)

In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 20
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 0.9589 - categorical_accuracy: 0.6168Source path =  /content/Project_data/val ; batch size = 20

Epoch 00001: saving model to model_init_2021-12-2910_13_59.732121/model-00001-0.95892-0.61680-1.14136-0.57000.h5
67/67 [==============================] - 358s 5s/step - loss: 0.9589 - categorical_accuracy: 0.6168 - val_loss: 1.1414 - val_categorical_accuracy: 0.5700 - lr: 5.0000e-04
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.1246 - categorical_accuracy: 0.5373
Epoch 00002: saving model to model_init_2021-12-2910_13_59.732121/model-00002-1.12460-0.53731-1.11825-0.56500.h5
67/67 [==============================] - 165s 2s/step - loss: 1.1246 - categorical_accuracy: 0.5373 - val_loss: 1.1182 - val_categorical_accuracy: 0.5650 - lr: 5.0000e-04
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 0.9623 - categorical_accuracy: 0

### Changing Batch size  to 30 Experiment [ 3]


In [ ]:
train_generator = generator_ex(train_path, train_doc, 30)
val_generator = generator_ex(val_path, val_doc, 30)

In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 30
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 0.4682 - categorical_accuracy: 0.8201Source path =  /content/Project_data/val ; batch size = 30
Batch:  4 Index: 30

Epoch 00001: saving model to model_init_2021-12-2910_13_59.732121/model-00001-0.46819-0.82013-1.13353-0.68750.h5
67/67 [==============================] - 344s 5s/step - loss: 0.4682 - categorical_accuracy: 0.8201 - val_loss: 1.1335 - val_categorical_accuracy: 0.6875 - lr: 6.2500e-05
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 0.4673 - categorical_accuracy: 0.8209
Epoch 00002: saving model to model_init_2021-12-2910_13_59.732121/model-00002-0.46729-0.82090-1.11419-0.68000.h5
67/67 [==============================] - 113s 2s/step - loss: 0.4673 - categorical_accuracy: 0.8209 - val_loss: 1.1142 - val_categorical_accuracy: 0.6800 - lr: 6.2500e-05
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 0.4819 - cat

### Changing Batch size  to 40 Experiment [ 4]


In [ ]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 40
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 0.3565 - categorical_accuracy: 0.8655Source path =  /content/Project_data/val ; batch size = 40
Batch:  3 Index: 40

Epoch 00001: saving model to model_init_2021-12-2910_13_59.732121/model-00001-0.35651-0.86551-1.13510-0.69583.h5
67/67 [==============================] - 726s 11s/step - loss: 0.3565 - categorical_accuracy: 0.8655 - val_loss: 1.1351 - val_categorical_accuracy: 0.6958 - lr: 7.8125e-06
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 0.3709 - categorical_accuracy: 0.8652
Epoch 00002: saving model to model_init_2021-12-2910_13_59.732121/model-00002-0.37087-0.86524-1.13143-0.69500.h5
67/67 [==============================] - 503s 8s/step - loss: 0.3709 - categorical_accuracy: 0.8652 - val_loss: 1.1314 - val_categorical_accuracy: 0.6950 - lr: 7.8125e-06
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 0.3819 - ca

### Change Optimizer to Adadelta Experiment [5]

In [ ]:
model_b.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_8 (Conv3D)           (None, 30, 60, 60, 32)    2624      
                                                                 
 activation_8 (Activation)   (None, 30, 60, 60, 32)    0         
                                                                 
 conv3d_9 (Conv3D)           (None, 30, 60, 60, 32)    27680     
                                                                 
 activation_9 (Activation)   (None, 30, 60, 60, 32)    0         
                                                                 
 max_pooling3d_6 (MaxPooling  (None, 10, 20, 20, 32)   0         
 3D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 10, 20, 20, 32)    0         
                                                      

In [ ]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 40
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 0.3430 - categorical_accuracy: 0.8759Source path =  /content/Project_data/val ; batch size = 40
Batch:  3 Index: 40

Epoch 00001: saving model to model_init_2021-12-2910_13_59.732121/model-00001-0.34299-0.87594-1.07855-0.71667.h5
67/67 [==============================] - 741s 11s/step - loss: 0.3430 - categorical_accuracy: 0.8759 - val_loss: 1.0785 - val_categorical_accuracy: 0.7167 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 0.3435 - categorical_accuracy: 0.8773
Epoch 00002: saving model to model_init_2021-12-2910_13_59.732121/model-00002-0.34352-0.87725-1.13587-0.71000.h5
67/67 [==============================] - 520s 8s/step - loss: 0.3435 - categorical_accuracy: 0.8773 - val_loss: 1.1359 - val_categorical_accuracy: 0.7100 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 0.3424 - categorica

### Change epoch to 20 Experiment [6]

In [ ]:
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# epochs = 20


In [ ]:
import keras
model_b.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_12 (Conv3D)          (None, 30, 60, 60, 32)    2624      
                                                                 
 activation_12 (Activation)  (None, 30, 60, 60, 32)    0         
                                                                 
 conv3d_13 (Conv3D)          (None, 30, 60, 60, 32)    27680     
                                                                 
 activation_13 (Activation)  (None, 30, 60, 60, 32)    0         
                                                                 
 max_pooling3d_10 (MaxPoolin  (None, 10, 20, 20, 32)   0         
 g3D)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 10, 20, 20, 32)    0         
                                                      

In [ ]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 40
Epoch 1/20
67/67 [==============================] - ETA: 0s - loss: 1.6223 - categorical_accuracy: 0.2035Source path =  /content/Project_data/val ; batch size = 40
Batch:  3 Index: 40

Epoch 00001: saving model to model_init_2022-01-0420_33_47.813370/model-00001-1.62229-0.20348-1.60949-0.25000.h5
67/67 [==============================] - 727s 11s/step - loss: 1.6223 - categorical_accuracy: 0.2035 - val_loss: 1.6095 - val_categorical_accuracy: 0.2500 - lr: 0.0010
Epoch 2/20
67/67 [==============================] - ETA: 0s - loss: 1.6123 - categorical_accuracy: 0.2077
Epoch 00002: saving model to model_init_2022-01-0420_33_47.813370/model-00002-1.61233-0.20773-1.60292-0.18000.h5
67/67 [==============================] - 517s 8s/step - loss: 1.6123 - categorical_accuracy: 0.2077 - val_loss: 1.6029 - val_categorical_accuracy: 0.1800 - lr: 0.0010
Epoch 3/20
67/67 [==============================] - ETA: 0s - loss: 1.6087 - categorica

### Model C Experiment [7]

In [ ]:
model_c = Sequential()
model_c.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding="same"))
model_c.add(Activation('relu'))
model_c.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Flatten())
model_c.add(Dense(512, activation='relu'))
model_c.add(BatchNormalization())
model_c.add(Dropout(0.5))
model_c.add(Dense(classes, activation='softmax'))
model_c.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_c.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_16 (Conv3D)          (None, 30, 60, 60, 32)    2624      
                                                                 
 activation_16 (Activation)  (None, 30, 60, 60, 32)    0         
                                                                 
 conv3d_17 (Conv3D)          (None, 30, 60, 60, 32)    27680     
                                                                 
 activation_17 (Activation)  (None, 30, 60, 60, 32)    0         
                                                                 
 max_pooling3d_12 (MaxPoolin  (None, 10, 20, 20, 32)   0         
 g3D)                                                            
                                                                 
 dropout_10 (Dropout)        (None, 10, 20, 20, 32)    0         
                                                      

### Model D Experiment [8]

In [ ]:
input_shape=(x,y,z,channel)

nb_filters = [8,16,32,64]
nb_dense = [1000, 500, 5]
# Define model
model_d = Sequential()

model_d.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_d.add(Flatten())

model_d.add(Dense(nb_dense[0], activation='relu'))
model_d.add(Dropout(0.5))

model_d.add(Dense(nb_dense[1], activation='relu'))
model_d.add(Dropout(0.5))

#softmax layer
model_d.add(Dense(nb_dense[2], activation='softmax'))
model_d.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_d.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_28 (Conv3D)          (None, 30, 64, 64, 8)     224       
                                                                 
 batch_normalization_10 (Bat  (None, 30, 64, 64, 8)    32        
 chNormalization)                                                
                                                                 
 activation_28 (Activation)  (None, 30, 64, 64, 8)     0         
                                                                 
 max_pooling3d_18 (MaxPoolin  (None, 15, 32, 32, 8)    0         
 g3D)                                                            
                                                                 
 conv3d_29 (Conv3D)          (None, 15, 32, 32, 16)    3472      
                                                                 
 batch_normalization_11 (Bat  (None, 15, 32, 32, 16)  

In [ ]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_d.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 2.8209 - categorical_accuracy: 0.2157Source path =  /content/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-01-0414_50_06.200503/model-00001-2.82091-0.21569-1.60420-0.24000.h5
67/67 [==============================] - 281s 4s/step - loss: 2.8209 - categorical_accuracy: 0.2157 - val_loss: 1.6042 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 2.4710 - categorical_accuracy: 0.2736
Epoch 00002: saving model to model_init_2022-01-0414_50_06.200503/model-00002-2.47101-0.27363-1.60877-0.22000.h5
67/67 [==============================] - 113s 2s/step - loss: 2.4710 - categorical_accuracy: 0.2736 - val_loss: 1.6088 - val_categorical_accuracy: 0.2200 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 2.4529 - categorical_accuracy: 0.2040
Ep

### Changing Model with images per frame, width and height of the images Experiment [9]

In [ ]:
x = 30 # number of frames
y = 64 # image width
z = 64 # image height 

classes = 5
channel = 1

In [ ]:
model_e = Sequential()
model_e.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding="same"))
model_e.add(Activation('relu'))
model_e.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Flatten())
model_e.add(Dense(512, activation='relu'))
model_e.add(BatchNormalization())
model_e.add(Dropout(0.5))
model_e.add(Dense(classes, activation='softmax'))
model_e.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_e.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_22 (Conv3D)          (None, 30, 64, 64, 32)    896       
                                                                 
 activation_22 (Activation)  (None, 30, 64, 64, 32)    0         
                                                                 
 conv3d_23 (Conv3D)          (None, 30, 64, 64, 32)    27680     
                                                                 
 activation_23 (Activation)  (None, 30, 64, 64, 32)    0         
                                                                 
 max_pooling3d_15 (MaxPoolin  (None, 10, 22, 22, 32)   0         
 g3D)                                                            
                                                                 
 dropout_14 (Dropout)        (None, 10, 22, 22, 32)    0         
                                                      

In [ ]:
batch_size = 5
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 20
model_e.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 5
Epoch 1/20
67/67 [==============================] - ETA: 0s - loss: 2.0158 - categorical_accuracy: 0.2149Source path =  /content/Project_data/val ; batch size = 5

Epoch 00001: saving model to model_init_2022-01-0414_50_06.200503/model-00001-2.01576-0.21493-1.61631-0.08000.h5
67/67 [==============================] - 145s 2s/step - loss: 2.0158 - categorical_accuracy: 0.2149 - val_loss: 1.6163 - val_categorical_accuracy: 0.0800 - lr: 0.0010
Epoch 2/20
67/67 [==============================] - ETA: 0s - loss: 2.1130 - categorical_accuracy: 0.1722
Epoch 00002: saving model to model_init_2022-01-0414_50_06.200503/model-00002-2.11298-0.17221-1.60894-0.24000.h5
67/67 [==============================] - 141s 2s/step - loss: 2.1130 - categorical_accuracy: 0.1722 - val_loss: 1.6089 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 3/20
67/67 [==============================] - ETA: 0s - loss: 2.0422 - categorical_accuracy: 0.1592
Epoc

### Model F Experiment [10]

In [ ]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 1
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,channel)

# Define model
model_f = Sequential()

model_f.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_f.add(Flatten())

model_f.add(Dense(nb_dense[0], activation='relu'))
model_f.add(Dropout(0.5))

model_f.add(Dense(nb_dense[1], activation='relu'))
model_f.add(Dropout(0.5))

#softmax layer
model_f.add(Dense(nb_dense[2], activation='softmax'))
model_f.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_f.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_32 (Conv3D)          (None, 30, 120, 120, 8)   224       
                                                                 
 batch_normalization_14 (Bat  (None, 30, 120, 120, 8)  32        
 chNormalization)                                                
                                                                 
 activation_32 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_22 (MaxPoolin  (None, 15, 60, 60, 8)    0         
 g3D)                                                            
                                                                 
 conv3d_33 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_15 (Bat  (None, 15, 60, 60, 16)  

In [ ]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 30
model_f.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 10
Epoch 1/30
67/67 [==============================] - ETA: 0s - loss: 3.0633 - categorical_accuracy: 0.2112Source path =  /content/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-01-0414_50_06.200503/model-00001-3.06329-0.21116-1.64008-0.20000.h5
67/67 [==============================] - 256s 4s/step - loss: 3.0633 - categorical_accuracy: 0.2112 - val_loss: 1.6401 - val_categorical_accuracy: 0.2000 - lr: 0.0010
Epoch 2/30
67/67 [==============================] - ETA: 0s - loss: 3.2528 - categorical_accuracy: 0.2338
Epoch 00002: saving model to model_init_2022-01-0414_50_06.200503/model-00002-3.25283-0.23383-1.64901-0.23000.h5
67/67 [==============================] - 108s 2s/step - loss: 3.2528 - categorical_accuracy: 0.2338 - val_loss: 1.6490 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 3/30
67/67 [==============================] - ETA: 0s - loss: 3.1822 - categorical_accuracy: 0.1791
Ep

### Model G Experiment [11]

In [ ]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,channel)

# Define model
model_g = Sequential()

model_g.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_g.add(Flatten())

model_g.add(Dense(nb_dense[0], activation='relu'))
model_g.add(Dropout(0.5))

model_g.add(Dense(nb_dense[1], activation='relu'))
model_g.add(Dropout(0.5))

#softmax layer
model_g.add(Dense(nb_dense[2], activation='softmax'))
model_g.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_g.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_36 (Conv3D)          (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_18 (Bat  (None, 30, 120, 120, 8)  32        
 chNormalization)                                                
                                                                 
 activation_36 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_26 (MaxPoolin  (None, 15, 60, 60, 8)    0         
 g3D)                                                            
                                                                 
 conv3d_37 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_19 (Bat  (None, 15, 60, 60, 16)  

In [ ]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_g.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 3.1865 - categorical_accuracy: 0.1916Source path =  /content/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-01-0414_50_06.200503/model-00001-3.18650-0.19155-1.62603-0.20000.h5
67/67 [==============================] - 271s 4s/step - loss: 3.1865 - categorical_accuracy: 0.1916 - val_loss: 1.6260 - val_categorical_accuracy: 0.2000 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 2.9829 - categorical_accuracy: 0.2587
Epoch 00002: saving model to model_init_2022-01-0414_50_06.200503/model-00002-2.98295-0.25871-1.63767-0.26000.h5
67/67 [==============================] - 116s 2s/step - loss: 2.9829 - categorical_accuracy: 0.2587 - val_loss: 1.6377 - val_categorical_accuracy: 0.2600 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 2.7639 - categorical_accuracy: 0.2239
Ep

### Model H Experiment [12]

In [ ]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3

input_shape=(x,y,z,channel)

# Define model
model_h = Sequential()

model_h.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_h.add(Flatten())

model_h.add(Dense(nb_dense[0], activation='relu'))
model_h.add(Dropout(0.5))

model_h.add(Dense(nb_dense[1], activation='relu'))
model_h.add(Dropout(0.5))

#softmax layer
model_h.add(Dense(nb_dense[2], activation='softmax'))
model_h.compile(optimizer=tf.keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_h.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_40 (Conv3D)          (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_22 (Bat  (None, 30, 120, 120, 8)  32        
 chNormalization)                                                
                                                                 
 activation_40 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_30 (MaxPoolin  (None, 15, 60, 60, 8)    0         
 g3D)                                                            
                                                                 
 conv3d_41 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_23 (Bat  (None, 15, 60, 60, 16)  

In [ ]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_h.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 2.8778 - categorical_accuracy: 0.2021Source path =  /content/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-01-0414_50_06.200503/model-00001-2.87776-0.20211-1.63510-0.17000.h5
67/67 [==============================] - 308s 4s/step - loss: 2.8778 - categorical_accuracy: 0.2021 - val_loss: 1.6351 - val_categorical_accuracy: 0.1700 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 2.9993 - categorical_accuracy: 0.2090
Epoch 00002: saving model to model_init_2022-01-0414_50_06.200503/model-00002-2.99932-0.20896-1.66648-0.21000.h5
67/67 [==============================] - 112s 2s/step - loss: 2.9993 - categorical_accuracy: 0.2090 - val_loss: 1.6665 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 3.0424 - categorical_accuracy: 0.1791
Ep

### Model I Experiment [13]

In [45]:
nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(30,120,120,3)

# Define model
model_i = Sequential()

model_i.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_i.add(BatchNormalization())
model_i.add(Activation('relu'))

model_i.add(MaxPooling3D(pool_size=(2,2,2)))

model_i.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_i.add(BatchNormalization())
model_i.add(Activation('relu'))

model_i.add(MaxPooling3D(pool_size=(2,2,2)))

model_i.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_i.add(BatchNormalization())
model_i.add(Activation('relu'))

model_i.add(MaxPooling3D(pool_size=(2,2,2)))

model_i.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_i.add(Activation('relu'))
model_i.add(Dropout(0.25))

model_i.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_i.add(Flatten())

model_i.add(Dense(nb_dense[0], activation='relu'))
model_i.add(Dropout(0.25))

model_i.add(Dense(nb_dense[1], activation='relu'))
model_i.add(Dropout(0.25))

#softmax layer
model_i.add(Dense(nb_dense[2], activation='softmax'))

In [46]:
optimiser = tf.keras.optimizers.Adam() #write your optimizer
model_i.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_i.summary())

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_31 (Conv3D)          (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_48 (Bat  (None, 30, 120, 120, 8)  32        
 chNormalization)                                                
                                                                 
 activation_31 (Activation)  (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_31 (MaxPoolin  (None, 15, 60, 60, 8)    0         
 g3D)                                                            
                                                                 
 conv3d_32 (Conv3D)          (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_49 (Bat  (None, 15, 60, 60, 16) 

In [39]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [40]:
batch_size = 32
num_epochs = 10
model_i.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 32
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.4594 - categorical_accuracy: 0.4069Source path =  /content/Project_data/val ; batch size = 32
Batch:  4 Index: 32

Epoch 00001: saving model to model_init_2022-01-0509_46_52.560445/model-00001-1.45936-0.40691-1.33915-0.41129.h5
67/67 [==============================] - 605s 9s/step - loss: 1.4594 - categorical_accuracy: 0.4069 - val_loss: 1.3392 - val_categorical_accuracy: 0.4113 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 0.9479 - categorical_accuracy: 0.6249
Epoch 00002: saving model to model_init_2022-01-0509_46_52.560445/model-00002-0.94791-0.62489-1.44426-0.35000.h5
67/67 [==============================] - 412s 6s/step - loss: 0.9479 - categorical_accuracy: 0.6249 - val_loss: 1.4443 - val_categorical_accuracy: 0.3500 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 0.7688 - categorical

### Final Model

In [15]:
nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(30,120,120,3)

# Define model
model_final = Sequential()

model_final.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(Activation('relu'))
model_final.add(Dropout(0.25))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_final.add(Flatten())

model_final.add(Dense(nb_dense[0], activation='relu'))
model_final.add(Dropout(0.5))

model_final.add(Dense(nb_dense[1], activation='relu'))
model_final.add(Dropout(0.5))

#softmax layer
model_final.add(Dense(nb_dense[2], activation='softmax'))

In [16]:
optimiser = tf.keras.optimizers.Adam() #write your optimizer
model_final.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_final.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_4 (Batc  (None, 30, 120, 120, 8)  32        
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 15, 60, 60, 8)    0         
 3D)                                                             
                                                                 
 conv3d_5 (Conv3D)           (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_5 (Batc  (None, 15, 60, 60, 16)  

In [17]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [ ]:
batch_size = 10
num_epochs = 20
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 10
Epoch 1/20
67/67 [==============================] - ETA: 0s - loss: 1.8812 - categorical_accuracy: 0.2594Source path =  /content/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2022-01-0420_33_47.813370/model-00001-1.88119-0.25943-1.57367-0.32000.h5
67/67 [==============================] - 272s 4s/step - loss: 1.8812 - categorical_accuracy: 0.2594 - val_loss: 1.5737 - val_categorical_accuracy: 0.3200 - lr: 0.0010
Epoch 2/20
67/67 [==============================] - ETA: 0s - loss: 1.5355 - categorical_accuracy: 0.3532
Epoch 00002: saving model to model_init_2022-01-0420_33_47.813370/model-00002-1.53549-0.35323-1.56012-0.27000.h5
67/67 [==============================] - 112s 2s/step - loss: 1.5355 - categorical_accuracy: 0.3532 - val_loss: 1.5601 - val_categorical_accuracy: 0.2700 - lr: 0.0010
Epoch 3/20
67/67 [==============================] - ETA: 0s - loss: 1.4821 - categorical_accuracy: 0.3234
Ep

In [ ]:
batch_size = 32
num_epochs = 20
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
67/67 [==============================] - ETA: 0s - loss: 0.9390 - categorical_accuracy: 0.6318
Epoch 00001: saving model to model_init_2022-01-0420_33_47.813370/model-00001-0.93899-0.63184-0.88141-0.62000.h5
67/67 [==============================] - 111s 2s/step - loss: 0.9390 - categorical_accuracy: 0.6318 - val_loss: 0.8814 - val_categorical_accuracy: 0.6200 - lr: 5.0000e-04
Epoch 2/20
67/67 [==============================] - ETA: 0s - loss: 0.9405 - categorical_accuracy: 0.6318
Epoch 00002: saving model to model_init_2022-01-0420_33_47.813370/model-00002-0.94050-0.63184-0.94192-0.56000.h5
67/67 [==============================] - 114s 2s/step - loss: 0.9405 - categorical_accuracy: 0.6318 - val_loss: 0.9419 - val_categorical_accuracy: 0.5600 - lr: 5.0000e-04
Epoch 3/20
67/67 [==============================] - ETA: 0s - loss: 0.9060 - categorical_accuracy: 0.6468
Epoch 00003: saving model to model_init_2022-01-0420_33_47.813370/model-00003-0.90602-0.64677-0.87930-0.63000.h5
6

In [ ]:
batch_size = 64
num_epochs = 20

train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)


model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 64
Epoch 1/20
67/67 [==============================] - ETA: 0s - loss: 0.4572 - categorical_accuracy: 0.8347Source path =  /content/Project_data/val ; batch size = 64
Batch:  2 Index: 64
Batch:  3 Index: 36
Batch:  4 Index: 28

Epoch 00001: saving model to model_init_2022-01-0420_33_47.813370/model-00001-0.45716-0.83469-0.72856-0.68354.h5
67/67 [==============================] - 789s 12s/step - loss: 0.4572 - categorical_accuracy: 0.8347 - val_loss: 0.7286 - val_categorical_accuracy: 0.6835 - lr: 3.1250e-05
Epoch 2/20
67/67 [==============================] - ETA: 0s - loss: 0.4205 - categorical_accuracy: 0.8482Batch:  7 Index: 16

Epoch 00002: saving model to model_init_2022-01-0420_33_47.813370/model-00002-0.42053-0.84822-0.75588-0.64773.h5
67/67 [==============================] - 441s 7s/step - loss: 0.4205 - categorical_accuracy: 0.8482 - val_loss: 0.7559 - val_categorical_accuracy: 0.6477 - lr: 3.1250e-05
Epoch 3/20
67/67 [=

In [18]:
batch_size = 40
num_epochs = 20

train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)


model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 40
Epoch 1/20
67/67 [==============================] - ETA: 0s - loss: 1.6646 - categorical_accuracy: 0.3223Source path =  /content/Project_data/val ; batch size = 40
Batch:  3 Index: 40

Epoch 00001: saving model to model_init_2022-01-0509_46_52.560445/model-00001-1.66456-0.32232-1.48050-0.42083.h5
67/67 [==============================] - 709s 11s/step - loss: 1.6646 - categorical_accuracy: 0.3223 - val_loss: 1.4805 - val_categorical_accuracy: 0.4208 - lr: 0.0010
Epoch 2/20
67/67 [==============================] - ETA: 0s - loss: 1.2141 - categorical_accuracy: 0.4747
Epoch 00002: saving model to model_init_2022-01-0509_46_52.560445/model-00002-1.21407-0.47468-1.13092-0.62500.h5
67/67 [==============================] - 487s 7s/step - loss: 1.2141 - categorical_accuracy: 0.4747 - val_loss: 1.1309 - val_categorical_accuracy: 0.6250 - lr: 0.0010
Epoch 3/20
67/67 [==============================] - ETA: 0s - loss: 1.0642 - categorica

###2D Model with RNN Experiment 15

In [24]:
from keras.layers.recurrent import LSTM


In [47]:
#nb_filters = [8,16,32,64]
#nb_dense = [256, 128, 5]

lstm_cells=128
dense_neurons=128
dropout=0.25

# Input
input_shape=(30,120,120,3)

model_cnnrnn = Sequential()

model_cnnrnn.add(TimeDistributed(Conv2D(8, (3, 3) , padding='same', activation='relu'),
                                  input_shape=input_shape))
model_cnnrnn.add(TimeDistributed(BatchNormalization()))
model_cnnrnn.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_cnnrnn.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu')))
model_cnnrnn.add(TimeDistributed(BatchNormalization()))
model_cnnrnn.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_cnnrnn.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model_cnnrnn.add(TimeDistributed(BatchNormalization()))
model_cnnrnn.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_cnnrnn.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model_cnnrnn.add(TimeDistributed(BatchNormalization()))
model_cnnrnn.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_cnnrnn.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model_cnnrnn.add(TimeDistributed(BatchNormalization()))
model_cnnrnn.add(TimeDistributed(MaxPooling2D((2, 2))))
        

model_cnnrnn.add(TimeDistributed(Flatten()))


model_cnnrnn.add(LSTM(lstm_cells))
model_cnnrnn.add(Dropout(dropout))
        
model_cnnrnn.add(Dense(dense_neurons,activation='relu'))
model_cnnrnn.add(Dropout(dropout))
        
model_cnnrnn.add(Dense(5, activation='softmax'))

In [48]:
optimiser = tf.keras.optimizers.Adam() #write your optimizer
model_cnnrnn.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_cnnrnn.summary())

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_80 (TimeDi  (None, 30, 120, 120, 8)  224       
 stributed)                                                      
                                                                 
 time_distributed_81 (TimeDi  (None, 30, 120, 120, 8)  32        
 stributed)                                                      
                                                                 
 time_distributed_82 (TimeDi  (None, 30, 60, 60, 8)    0         
 stributed)                                                      
                                                                 
 time_distributed_83 (TimeDi  (None, 30, 60, 60, 16)   1168      
 stributed)                                                      
                                                                 
 time_distributed_84 (TimeDi  (None, 30, 60, 60, 16) 

In [ ]:
batch_size = 32
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_cnnrnn.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 32
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.2841 - categorical_accuracy: 0.4457Source path =  /content/Project_data/val ; batch size = 32
Batch:  4 Index: 32

Epoch 00001: saving model to model_init_2022-01-0509_46_52.560445/model-00001-1.28415-0.44572-2.02988-0.22581.h5
67/67 [==============================] - 616s 9s/step - loss: 1.2841 - categorical_accuracy: 0.4457 - val_loss: 2.0299 - val_categorical_accuracy: 0.2258 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.0222 - categorical_accuracy: 0.5925
Epoch 00002: saving model to model_init_2022-01-0509_46_52.560445/model-00002-1.02223-0.59250-3.67499-0.15000.h5
67/67 [==============================] - 426s 6s/step - loss: 1.0222 - categorical_accuracy: 0.5925 - val_loss: 3.6750 - val_categorical_accuracy: 0.1500 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 0.9407 - categorical